In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.config import list_physical_devices
from tensorflow.image import ResizeMethod
from tensorflow.image import resize_with_pad

import utils

In [2]:
trainX, trainY, testX, testY = utils.load_dataset()

In [3]:
def ensemble_logreg():
    inputs = keras.Input(shape=(32,32,3))

    all_models = utils.get_models('models')
    all_model_output = []

    counter = 0
    for m in all_models:
        m.trainable = False
        all_model_output.append(m(inputs))
        print(str(counter) + ' done')
        counter += 1
    
    x = layers.concatenate(all_model_output)

    outputs = layers.Dense(10, activation='softmax')(x)
    
    ensemble_model = keras.Model(inputs=inputs, outputs=outputs, name='ensemble_logreg')
    ensemble_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
    return ensemble_model

In [4]:
model = ensemble_logreg()
model.summary()

0 done
1 done
2 done
3 done
Model: "ensemble_logreg"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
custom_2 (Functional)           (None, 10)           924106      input_1[0][0]                    
__________________________________________________________________________________________________
custom_1 (Functional)           (None, 10)           2524938     input_1[0][0]                    
__________________________________________________________________________________________________
transfer_densenet (Functional)  (None, 10)           7347338     input_1[0][0]                    
________________________________________________________

In [5]:
callback = keras.callbacks.ModelCheckpoint('models/ensemble_logreg', monitor='val_loss', save_best_only=True)

In [6]:
history = model.fit(x=trainX, y=trainY, batch_size=512, epochs=50, callbacks=callback, validation_data=(testX, testY))

Epoch 1/50
98/98 [==============================] - 608s 4s/step - loss: 2.3257 - categorical_accuracy: 0.1442 - auc: 0.5219 - val_loss: 1.9892 - val_categorical_accuracy: 0.3791 - val_auc: 0.8425
INFO:tensorflow:Assets written to: models/ensemble_logreg/assets
Epoch 2/50
98/98 [==============================] - 26s 268ms/step - loss: 1.8520 - categorical_accuracy: 0.5163 - auc: 0.9145 - val_loss: 1.6019 - val_categorical_accuracy: 0.7527 - val_auc: 0.9549
INFO:tensorflow:Assets written to: models/ensemble_logreg/assets
Epoch 3/50
98/98 [==============================] - 26s 266ms/step - loss: 1.4475 - categorical_accuracy: 0.8458 - auc: 0.9839 - val_loss: 1.2945 - val_categorical_accuracy: 0.8249 - val_auc: 0.9741
INFO:tensorflow:Assets written to: models/ensemble_logreg/assets
Epoch 4/50
98/98 [==============================] - 27s 271ms/step - loss: 1.1313 - categorical_accuracy: 0.9224 - auc: 0.9935 - val_loss: 1.0663 - val_categorical_accuracy: 0.8405 - val_auc: 0.9805
INFO:tensor